In [1]:
## This file implements neural networks and random forest on p0006presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 69.23% for over-sampling data after improvement, 
## 93.33% for combination data and 88.89% with dropout.
## For random forest, the accuracy is 65.81% for over-sampling data and 95.56% for combination data.
## For random forest with cross-validation, the mean accuracy is 66.35% for over-sampling, and 96.89% for combination data.

In [22]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [23]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0006presabs_qual.csv')
df.shape

(255, 86)

In [24]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [25]:
df['pheno']

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      1
10     0
11     2
12     1
13     0
14     0
15     0
16     1
17     1
18     2
19     0
20     1
21     1
22     0
23     0
24     2
25     0
26     2
27     0
28     0
29     2
      ..
225    1
226    1
227    1
228    2
229    2
230    1
231    0
232    1
233    1
234    1
235    2
236    0
237    0
238    0
239    0
240    0
241    1
242    0
243    0
244    1
245    0
246    1
247    0
248    0
249    0
250    1
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [26]:
df['pheno'].value_counts()

0    130
1     86
2     39
Name: pheno, dtype: int64

In [27]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [28]:
df_clean = df.drop(columns=['id'])

In [29]:
df_clean.shape

(255, 85)

In [30]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [31]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 84) (255,)


In [32]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 130), (1, 130), (2, 130)]


In [44]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 68), (1, 12), (2, 67)]


In [33]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [45]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [46]:
############# Fully-Connected Neural Network ################

In [47]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [48]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [49]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/100
273/273 [==============================] - 0s 389us/step - loss: 1.1499 - accuracy: 0.3333 - val_loss: 1.1307 - val_accuracy: 0.3675
Epoch 2/100
273/273 [==============================] - 0s 89us/step - loss: 1.0383 - accuracy: 0.4505 - val_loss: 1.0534 - val_accuracy: 0.4872
Epoch 3/100
273/273 [==============================] - 0s 99us/step - loss: 0.9717 - accuracy: 0.5531 - val_loss: 1.0076 - val_accuracy: 0.4786
Epoch 4/100
273/273 [==============================] - 0s 99us/step - loss: 0.9273 - accuracy: 0.6081 - val_loss: 0.9774 - val_accuracy: 0.5556
Epoch 5/100
273/273 [==============================] - 0s 98us/step - loss: 0.8995 - accuracy: 0.6300 - val_loss: 0.9598 - val_accuracy: 0.5470
Epoch 6/100
273/273 [==============================] - 0s 105us/step - loss: 0.8730 - accuracy: 0.6264 - val_loss: 0.9398 - val_accuracy: 0.5556
Epoch 7/100
273/273 [==============================] - 0s 100us/step - loss: 0.8501 - ac

Epoch 57/100
273/273 [==============================] - 0s 128us/step - loss: 0.5836 - accuracy: 0.7363 - val_loss: 0.7773 - val_accuracy: 0.6496
Epoch 58/100
273/273 [==============================] - 0s 112us/step - loss: 0.5823 - accuracy: 0.7473 - val_loss: 0.7727 - val_accuracy: 0.6496
Epoch 59/100
273/273 [==============================] - 0s 107us/step - loss: 0.5778 - accuracy: 0.7436 - val_loss: 0.7717 - val_accuracy: 0.6496
Epoch 60/100
273/273 [==============================] - 0s 100us/step - loss: 0.5789 - accuracy: 0.7546 - val_loss: 0.7678 - val_accuracy: 0.6496
Epoch 61/100
273/273 [==============================] - 0s 108us/step - loss: 0.5748 - accuracy: 0.7509 - val_loss: 0.7693 - val_accuracy: 0.6410
Epoch 62/100
273/273 [==============================] - 0s 110us/step - loss: 0.5702 - accuracy: 0.7509 - val_loss: 0.7683 - val_accuracy: 0.6410
Epoch 63/100
273/273 [==============================] - ETA: 0s - loss: 0.5345 - accuracy: 0.84 - 0s 132us/step - loss: 0.56

In [51]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

117/117 [==============================] - 0s 72us/step
over-sampling test accuracy: 66.67%


In [101]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [102]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [103]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 273 samples, validate on 117 samples
Epoch 1/1000
273/273 [==============================] - 0s 520us/step - loss: 1.0871 - accuracy: 0.3626 - val_loss: 1.0787 - val_accuracy: 0.3932
Epoch 2/1000
273/273 [==============================] - 0s 92us/step - loss: 1.0384 - accuracy: 0.5531 - val_loss: 1.0548 - val_accuracy: 0.5299
Epoch 3/1000
273/273 [==============================] - 0s 111us/step - loss: 1.0016 - accuracy: 0.6300 - val_loss: 1.0249 - val_accuracy: 0.5641
Epoch 4/1000
273/273 [==============================] - 0s 128us/step - loss: 0.9636 - accuracy: 0.6410 - val_loss: 0.9959 - val_accuracy: 0.5556
Epoch 5/1000
273/273 [==============================] - 0s 100us/step - loss: 0.9266 - accuracy: 0.6447 - val_loss: 0.9710 - val_accuracy: 0.5556
Epoch 6/1000
273/273 [==============================] - 0s 97us/step - loss: 0.8930 - accuracy: 0.6374 - val_loss: 0.9509 - val_accuracy: 0.5641
Epoch 7/1000
273/273 [==============================] - 0s 111us/step - loss: 0.

273/273 [==============================] - 0s 119us/step - loss: 0.5010 - accuracy: 0.7839 - val_loss: 0.7591 - val_accuracy: 0.6752
Epoch 57/1000
273/273 [==============================] - 0s 104us/step - loss: 0.4975 - accuracy: 0.7949 - val_loss: 0.7807 - val_accuracy: 0.6752
Epoch 58/1000
273/273 [==============================] - 0s 109us/step - loss: 0.4997 - accuracy: 0.7985 - val_loss: 0.7788 - val_accuracy: 0.6838
Epoch 59/1000
273/273 [==============================] - 0s 87us/step - loss: 0.5011 - accuracy: 0.7692 - val_loss: 0.7610 - val_accuracy: 0.6325
Epoch 60/1000
273/273 [==============================] - 0s 93us/step - loss: 0.4905 - accuracy: 0.7949 - val_loss: 0.7950 - val_accuracy: 0.6667
Epoch 61/1000
273/273 [==============================] - 0s 94us/step - loss: 0.4953 - accuracy: 0.7912 - val_loss: 0.7473 - val_accuracy: 0.6838
Epoch 62/1000
273/273 [==============================] - 0s 99us/step - loss: 0.4979 - accuracy: 0.7692 - val_loss: 0.7745 - val_accura

273/273 [==============================] - 0s 85us/step - loss: 0.4260 - accuracy: 0.8022 - val_loss: 0.8029 - val_accuracy: 0.6581
Epoch 113/1000
273/273 [==============================] - 0s 87us/step - loss: 0.4164 - accuracy: 0.8205 - val_loss: 0.7970 - val_accuracy: 0.7350
Epoch 114/1000
273/273 [==============================] - 0s 86us/step - loss: 0.4167 - accuracy: 0.8315 - val_loss: 0.7984 - val_accuracy: 0.7009
Epoch 115/1000
273/273 [==============================] - 0s 92us/step - loss: 0.4217 - accuracy: 0.8095 - val_loss: 0.7904 - val_accuracy: 0.7009
Epoch 116/1000
273/273 [==============================] - 0s 82us/step - loss: 0.4286 - accuracy: 0.8022 - val_loss: 0.8127 - val_accuracy: 0.6752
Epoch 117/1000
273/273 [==============================] - 0s 99us/step - loss: 0.4107 - accuracy: 0.8205 - val_loss: 0.7850 - val_accuracy: 0.6923
Epoch 118/1000
273/273 [==============================] - 0s 75us/step - loss: 0.4147 - accuracy: 0.8242 - val_loss: 0.8109 - val_acc

Epoch 168/1000
273/273 [==============================] - 0s 86us/step - loss: 0.3746 - accuracy: 0.8425 - val_loss: 0.8209 - val_accuracy: 0.7009
Epoch 169/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3801 - accuracy: 0.8315 - val_loss: 0.8322 - val_accuracy: 0.7009
Epoch 170/1000
273/273 [==============================] - 0s 81us/step - loss: 0.3746 - accuracy: 0.8205 - val_loss: 0.8540 - val_accuracy: 0.6752
Epoch 171/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3787 - accuracy: 0.8278 - val_loss: 0.8278 - val_accuracy: 0.7009
Epoch 172/1000
273/273 [==============================] - 0s 86us/step - loss: 0.3753 - accuracy: 0.8315 - val_loss: 0.8435 - val_accuracy: 0.6752
Epoch 173/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3754 - accuracy: 0.8168 - val_loss: 0.8204 - val_accuracy: 0.7094
Epoch 174/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3716 - accuracy: 0.8278 - val_loss: 0

Epoch 224/1000
273/273 [==============================] - 0s 89us/step - loss: 0.3626 - accuracy: 0.8242 - val_loss: 0.8765 - val_accuracy: 0.7009
Epoch 225/1000
273/273 [==============================] - 0s 88us/step - loss: 0.3580 - accuracy: 0.8315 - val_loss: 0.8328 - val_accuracy: 0.7009
Epoch 226/1000
273/273 [==============================] - 0s 84us/step - loss: 0.3625 - accuracy: 0.8242 - val_loss: 0.8390 - val_accuracy: 0.7009
Epoch 227/1000
273/273 [==============================] - 0s 83us/step - loss: 0.3659 - accuracy: 0.8242 - val_loss: 0.8273 - val_accuracy: 0.7009
Epoch 228/1000
273/273 [==============================] - 0s 84us/step - loss: 0.3798 - accuracy: 0.8205 - val_loss: 0.8903 - val_accuracy: 0.6667
Epoch 229/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3605 - accuracy: 0.8315 - val_loss: 0.8211 - val_accuracy: 0.7350
Epoch 230/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3572 - accuracy: 0.8278 - val_loss: 0

...

Epoch 727/1000
273/273 [==============================] - 0s 84us/step - loss: 0.3330 - accuracy: 0.8315 - val_loss: 0.9537 - val_accuracy: 0.7009
Epoch 728/1000
273/273 [==============================] - 0s 81us/step - loss: 0.3353 - accuracy: 0.8352 - val_loss: 0.9623 - val_accuracy: 0.6923
Epoch 729/1000
273/273 [==============================] - 0s 81us/step - loss: 0.3402 - accuracy: 0.8059 - val_loss: 0.9428 - val_accuracy: 0.7179
Epoch 730/1000
273/273 [==============================] - 0s 91us/step - loss: 0.3355 - accuracy: 0.8242 - val_loss: 0.9701 - val_accuracy: 0.6752
Epoch 731/1000
273/273 [==============================] - 0s 92us/step - loss: 0.3352 - accuracy: 0.8278 - val_loss: 0.9550 - val_accuracy: 0.7009
Epoch 732/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3339 - accuracy: 0.8242 - val_loss: 0.9948 - val_accuracy: 0.6667
Epoch 733/1000
273/273 [==============================] - 0s 97us/step - loss: 0.3346 - accuracy: 0.8315 - val_loss: 0

Epoch 783/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3298 - accuracy: 0.8315 - val_loss: 0.9794 - val_accuracy: 0.7009
Epoch 784/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3388 - accuracy: 0.8278 - val_loss: 0.9951 - val_accuracy: 0.6923
Epoch 785/1000
273/273 [==============================] - 0s 92us/step - loss: 0.3323 - accuracy: 0.8315 - val_loss: 0.9561 - val_accuracy: 0.6923
Epoch 786/1000
273/273 [==============================] - 0s 87us/step - loss: 0.3339 - accuracy: 0.8205 - val_loss: 0.9369 - val_accuracy: 0.6923
Epoch 787/1000
273/273 [==============================] - 0s 78us/step - loss: 0.3427 - accuracy: 0.8388 - val_loss: 0.9955 - val_accuracy: 0.6667
Epoch 788/1000
273/273 [==============================] - 0s 91us/step - loss: 0.3371 - accuracy: 0.8278 - val_loss: 0.9465 - val_accuracy: 0.6923
Epoch 789/1000
273/273 [==============================] - 0s 87us/step - loss: 0.3322 - accuracy: 0.8205 - val_loss: 0

Epoch 839/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3318 - accuracy: 0.8278 - val_loss: 0.9758 - val_accuracy: 0.7009
Epoch 840/1000
273/273 [==============================] - 0s 88us/step - loss: 0.3333 - accuracy: 0.8278 - val_loss: 0.9713 - val_accuracy: 0.7009
Epoch 841/1000
273/273 [==============================] - 0s 80us/step - loss: 0.3292 - accuracy: 0.8352 - val_loss: 0.9541 - val_accuracy: 0.7009
Epoch 842/1000
273/273 [==============================] - 0s 80us/step - loss: 0.3378 - accuracy: 0.8242 - val_loss: 0.9733 - val_accuracy: 0.6667
Epoch 843/1000
273/273 [==============================] - 0s 76us/step - loss: 0.3353 - accuracy: 0.8278 - val_loss: 0.9656 - val_accuracy: 0.7009
Epoch 844/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3404 - accuracy: 0.8315 - val_loss: 0.9595 - val_accuracy: 0.6923
Epoch 845/1000
273/273 [==============================] - 0s 77us/step - loss: 0.3351 - accuracy: 0.8278 - val_loss: 0

Epoch 895/1000
273/273 [==============================] - 0s 93us/step - loss: 0.3310 - accuracy: 0.8388 - val_loss: 0.9663 - val_accuracy: 0.6923
Epoch 896/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3370 - accuracy: 0.8352 - val_loss: 0.9782 - val_accuracy: 0.6923
Epoch 897/1000
273/273 [==============================] - 0s 85us/step - loss: 0.3399 - accuracy: 0.8242 - val_loss: 1.0247 - val_accuracy: 0.6667
Epoch 898/1000
273/273 [==============================] - 0s 96us/step - loss: 0.3366 - accuracy: 0.8205 - val_loss: 0.9609 - val_accuracy: 0.7009
Epoch 899/1000
273/273 [==============================] - 0s 87us/step - loss: 0.3364 - accuracy: 0.8168 - val_loss: 0.9536 - val_accuracy: 0.6838
Epoch 900/1000
273/273 [==============================] - 0s 81us/step - loss: 0.3385 - accuracy: 0.8242 - val_loss: 1.0127 - val_accuracy: 0.6667
Epoch 901/1000
273/273 [==============================] - 0s 79us/step - loss: 0.3323 - accuracy: 0.8168 - val_loss: 0

273/273 [==============================] - 0s 79us/step - loss: 0.3332 - accuracy: 0.8352 - val_loss: 0.9662 - val_accuracy: 0.6923
Epoch 951/1000
273/273 [==============================] - 0s 91us/step - loss: 0.3318 - accuracy: 0.8168 - val_loss: 0.9522 - val_accuracy: 0.7179
Epoch 952/1000
273/273 [==============================] - 0s 106us/step - loss: 0.3297 - accuracy: 0.8242 - val_loss: 0.9614 - val_accuracy: 0.6667
Epoch 953/1000
273/273 [==============================] - 0s 95us/step - loss: 0.3297 - accuracy: 0.8352 - val_loss: 0.9785 - val_accuracy: 0.6667
Epoch 954/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3333 - accuracy: 0.8315 - val_loss: 0.9575 - val_accuracy: 0.6923
Epoch 955/1000
273/273 [==============================] - 0s 89us/step - loss: 0.3316 - accuracy: 0.8278 - val_loss: 0.9782 - val_accuracy: 0.6667
Epoch 956/1000
273/273 [==============================] - 0s 82us/step - loss: 0.3311 - accuracy: 0.8352 - val_loss: 0.9740 - val_ac

In [104]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

117/117 [==============================] - 0s 84us/step
over-sampling test accuracy: 69.23%


In [105]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [106]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [107]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 102 samples, validate on 45 samples
Epoch 1/100
102/102 [==============================] - 0s 947us/step - loss: 1.6157 - accuracy: 0.0882 - val_loss: 1.5211 - val_accuracy: 0.1111
Epoch 2/100
102/102 [==============================] - 0s 126us/step - loss: 1.4212 - accuracy: 0.0980 - val_loss: 1.3382 - val_accuracy: 0.2444
Epoch 3/100
102/102 [==============================] - 0s 143us/step - loss: 1.2615 - accuracy: 0.3039 - val_loss: 1.1860 - val_accuracy: 0.2444
Epoch 4/100
102/102 [==============================] - 0s 198us/step - loss: 1.1288 - accuracy: 0.2745 - val_loss: 1.0653 - val_accuracy: 0.2889
Epoch 5/100
102/102 [==============================] - 0s 180us/step - loss: 1.0212 - accuracy: 0.3431 - val_loss: 0.9656 - val_accuracy: 0.4444
Epoch 6/100
102/102 [==============================] - 0s 182us/step - loss: 0.9352 - accuracy: 0.5196 - val_loss: 0.8861 - val_accuracy: 0.6444
Epoch 7/100
102/102 [==============================] - 0s 212us/step - loss: 0.8602 -

Epoch 57/100
102/102 [==============================] - 0s 198us/step - loss: 0.2025 - accuracy: 0.9216 - val_loss: 0.2683 - val_accuracy: 0.8889
Epoch 58/100
102/102 [==============================] - 0s 162us/step - loss: 0.1985 - accuracy: 0.9216 - val_loss: 0.2673 - val_accuracy: 0.8889
Epoch 59/100
102/102 [==============================] - 0s 149us/step - loss: 0.1948 - accuracy: 0.9216 - val_loss: 0.2659 - val_accuracy: 0.8889
Epoch 60/100
102/102 [==============================] - 0s 173us/step - loss: 0.1914 - accuracy: 0.9216 - val_loss: 0.2641 - val_accuracy: 0.8889
Epoch 61/100
102/102 [==============================] - 0s 187us/step - loss: 0.1879 - accuracy: 0.9216 - val_loss: 0.2618 - val_accuracy: 0.8889
Epoch 62/100
102/102 [==============================] - 0s 161us/step - loss: 0.1841 - accuracy: 0.9216 - val_loss: 0.2594 - val_accuracy: 0.8889
Epoch 63/100
102/102 [==============================] - 0s 157us/step - loss: 0.1808 - accuracy: 0.9216 - val_loss: 0.2567 -

In [108]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

45/45 [==============================] - 0s 138us/step
combination test accuracy: 93.33%


In [109]:
#### add dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 102 samples, validate on 45 samples
Epoch 1/100
102/102 [==============================] - 0s 1ms/step - loss: 3.8577 - accuracy: 0.6471 - val_loss: 0.7763 - val_accuracy: 0.7556
Epoch 2/100
102/102 [==============================] - 0s 142us/step - loss: 3.0313 - accuracy: 0.6765 - val_loss: 0.7213 - val_accuracy: 0.7556
Epoch 3/100
102/102 [==============================] - 0s 184us/step - loss: 3.2879 - accuracy: 0.6569 - val_loss: 0.6762 - val_accuracy: 0.7556
Epoch 4/100
102/102 [==============================] - 0s 208us/step - loss: 3.9710 - accuracy: 0.6471 - val_loss: 0.6403 - val_accuracy: 0.7556
Epoch 5/100
102/102 [==============================] - 0s 201us/step - loss: 3.9125 - accuracy: 0.6373 - val_loss: 0.6094 - val_accuracy: 0.7556
Epoch 6/100
102/102 [==============================] - 0s 201us/step - loss: 3.5825 - accuracy: 0.6961 - val_loss: 0.5820 - val_accuracy: 0.7778
Epoch 7/100
102/102 [==============================] - 0s 192us/step - loss: 2.4390 - a

Epoch 57/100
102/102 [==============================] - 0s 194us/step - loss: 1.4089 - accuracy: 0.7353 - val_loss: 0.4763 - val_accuracy: 0.9111
Epoch 58/100
102/102 [==============================] - 0s 160us/step - loss: 1.4931 - accuracy: 0.7353 - val_loss: 0.4778 - val_accuracy: 0.8889
Epoch 59/100
102/102 [==============================] - 0s 159us/step - loss: 1.8274 - accuracy: 0.7255 - val_loss: 0.4759 - val_accuracy: 0.9111
Epoch 60/100
102/102 [==============================] - 0s 143us/step - loss: 1.3113 - accuracy: 0.7843 - val_loss: 0.4626 - val_accuracy: 0.9111
Epoch 61/100
102/102 [==============================] - 0s 143us/step - loss: 1.5416 - accuracy: 0.7451 - val_loss: 0.4511 - val_accuracy: 0.9111
Epoch 62/100
102/102 [==============================] - 0s 141us/step - loss: 1.2155 - accuracy: 0.7647 - val_loss: 0.4423 - val_accuracy: 0.8889
Epoch 63/100
102/102 [==============================] - 0s 164us/step - loss: 1.0708 - accuracy: 0.7353 - val_loss: 0.4251 -

In [112]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

45/45 [==============================] - 0s 182us/step
combination test accuracy: 88.89%


In [113]:
############## Random Forest ##############

In [114]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [115]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 65.81%


In [116]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [117]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 95.56%


In [118]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.61403509 0.68518519 0.61111111 0.68518519 0.72222222]
0.6635477582846004


In [119]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.95       1.         0.89473684]
0.9689473684210526
